In [3]:
%%bash
# 1. Install System Dependencies for OCR
sudo apt-get update -q
sudo apt-get install -y tesseract-ocr -q

# 2. Install Python Libraries
pip install -q \
    pdfplumber \
    sentence-transformers \
    chromadb \
    openai \
    pytesseract \
    pillow \
    tiktoken


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.4 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,491 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-secu

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.19.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requir

In [ ]:
import os
import sys
import uuid
import shutil
import pdfplumber
import pytesseract
import chromadb
from PIL import Image
from typing import List, Dict
from google.colab import drive
from google.colab import files
from sentence_transformers import SentenceTransformer, CrossEncoder
import openai

# CONFIGURATION

MOUNT_PATH = '/content/drive'
PROJECT_DIR = '/content/drive/MyDrive/colab_rag_data'
CHROMA_PATH = os.path.join(PROJECT_DIR, "chroma_db")
EMBED_MODEL_NAME = "all-MiniLM-L6-v2" # Fast and effective
CROSS_ENCODER_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"


# 1. SETUP & UTILITIES

def setup_environment():
    """Mounts drive and creates project folders."""
    if not os.path.exists(MOUNT_PATH):
        print("Mounting Google Drive...")
        drive.mount(MOUNT_PATH)

    if not os.path.exists(PROJECT_DIR):
        os.makedirs(PROJECT_DIR)
        print(f"Created project directory: {PROJECT_DIR}")
    else:
        print(f"Using project directory: {PROJECT_DIR}")

    # Get OpenAI Key safely
    print("\n--- API SETUP ---")
    api_key = input("Enter OpenAI API Key (press Enter to skip if testing retrieval only): ").strip()
    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
        openai.api_key = api_key
        print("API Key set.")
    else:
        print("No API Key provided. System will run in 'Retrieval Only' mode.")
    return api_key


# 2. PDF PROCESSING & OCR

def extract_text_from_pdf(pdf_path: str) -> List[Dict]:
    """Extracts text page-by-page. Uses OCR if text extraction fails."""
    print(f"Processing: {os.path.basename(pdf_path)}...")
    extracted_pages = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()

            # If text is empty or very short, try OCR
            if not text or len(text.strip()) < 50:
                print(f"  - Page {i}: Text empty, running OCR...")
                try:
                    # Render page to image for OCR
                    im = page.to_image(resolution=300).original
                    text = pytesseract.image_to_string(im)
                    is_ocr = True
                except Exception as e:
                    print(f"    OCR Failed for page {i}: {e}")
                    text = ""
                    is_ocr = False
            else:
                is_ocr = False

            if text.strip():
                extracted_pages.append({
                    "page": i,
                    "text": text,
                    "is_ocr": is_ocr,
                    "source": os.path.basename(pdf_path)
                })

    print(f"Extraction complete. Found {len(extracted_pages)} usable pages.")
    return extracted_pages

def chunk_text(pages: List[Dict], chunk_size=400, overlap=100) -> List[Dict]:
    """Splits pages into overlapping chunks."""
    chunks = []
    for p in pages:
        words = p['text'].split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk_words = words[i:i + chunk_size]
            chunk_text = " ".join(chunk_words)

            if len(chunk_text) > 50: # Skip tiny chunks
                chunks.append({
                    "id": str(uuid.uuid4()),
                    "text": chunk_text,
                    "metadata": {
                        "page": p['page'],
                        "source": p['source'],
                        "is_ocr": p['is_ocr']
                    }
                })
    print(f"Chunking complete. Created {len(chunks)} chunks.")
    return chunks


# 3. VECTOR DATABASE (CHROMA)

class VectorStore:
    def __init__(self, persist_path):
        print(f"Initializing Vector DB at {persist_path}...")
        self.client = chromadb.PersistentClient(path=persist_path)
        self.collection = self.client.get_or_create_collection(name="pdf_knowledge_base")
        self.embedder = SentenceTransformer(EMBED_MODEL_NAME)
        self.reranker = CrossEncoder(CROSS_ENCODER_NAME)

    def add_documents(self, chunks: List[Dict]):
        if not chunks: return

        # Prepare data for Chroma
        ids = [c['id'] for c in chunks]
        docs = [c['text'] for c in chunks]
        metas = [c['metadata'] for c in chunks]

        print("Generating embeddings (this may take a moment)...")
        embeddings = self.embedder.encode(docs, show_progress_bar=True).tolist()

        # Batch insert to avoid memory issues
        batch_size = 100
        for i in range(0, len(ids), batch_size):
            self.collection.add(
                ids=ids[i:i+batch_size],
                documents=docs[i:i+batch_size],
                metadatas=metas[i:i+batch_size],
                embeddings=embeddings[i:i+batch_size]
            )
        print("Data indexed successfully.")

    def query(self, query_text: str, top_k=10, rerank_top_n=5):
        # 1. Dense Retrieval
        q_emb = self.embedder.encode([query_text]).tolist()
        results = self.collection.query(
            query_embeddings=q_emb,
            n_results=top_k
        )

        # Flatten results
        hits = []
        if results['documents']:
            for i, doc in enumerate(results['documents'][0]):
                hits.append({
                    "text": doc,
                    "metadata": results['metadatas'][0][i],
                    "score": results['distances'][0][i]
                })

        # 2. Re-ranking (Refining relevance)
        if hits and rerank_top_n > 0:
            pairs = [[query_text, hit['text']] for hit in hits]
            scores = self.reranker.predict(pairs)
            for i, hit in enumerate(hits):
                hit['rerank_score'] = scores[i]

            # Sort by re-rank score (higher is better)
            hits = sorted(hits, key=lambda x: x['rerank_score'], reverse=True)
            hits = hits[:rerank_top_n]

        return hits


# 4. LLM GENERATION

def generate_answer(query: str, context_hits: List[Dict], api_key: str):
    """Constructs prompt and calls OpenAI."""
    if not api_key:
        return "No API Key provided. Skipping LLM generation."

    # Build context string with citations
    context_str = ""
    for hit in context_hits:
        meta = hit['metadata']
        context_str += f"[Source: {meta['source']}, Page: {meta['page']}]\n{hit['text']}\n\n"

    prompt = f"""
    You are a helpful research assistant. Use the provided context to answer the question.
    If the answer is not in the context, state that you don't know.
    Always cite the page numbers provided in the context blocks.

    CONTEXT:
    {context_str}

    QUESTION:
    {query}

    ANSWER:
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", # Switch to gpt-4 if you have access
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error calling OpenAI: {e}"


# 5. MAIN EXECUTION LOOP

def main():
    api_key = setup_environment()
    db = VectorStore(CHROMA_PATH)

    print("\n--- FILE UPLOAD ---")
    print("1. Upload new PDF")
    print("2. Use existing index (Skip upload)")
    choice = input("Enter choice (1 or 2): ")

    if choice == '1':
        uploaded = files.upload()
        if uploaded:
            filename = list(uploaded.keys())[0]
            # Process
            raw_pages = extract_text_from_pdf(filename)
            chunks = chunk_text(raw_pages)
            db.add_documents(chunks)

            # Optional: move file to Drive to keep it
            dest_path = os.path.join(PROJECT_DIR, filename)
            shutil.move(filename, dest_path)
            print(f"File saved to {dest_path}")

    print("\n" + "="*40)
    print("🤖 RAG CHATBOT READY")
    print("="*40)

    while True:
        query = input("\nAsk a question (or type 'quit'): ")
        if query.lower() in ['quit', 'exit']:
            break

        # Retrieve
        print("🔍 Searching...")
        hits = db.query(query)

        # Display Sources (for verification)
        print("\n--- Top Sources ---")
        for h in hits[:2]:
            print(f"[Page {h['metadata']['page']}] {h['text'][:150]}...")

        # Generate Answer
        print("\n--- Answer ---")
        if api_key:
            answer = generate_answer(query, hits, api_key)
            print(answer)
        else:
            print("No API Key. Review sources above.")

if __name__ == "__main__":
    main()
